In [ ]:
import requests
from urllib.request import urlopen
from io import StringIO
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pyproj import CRS, Transformer
import json

# Data Collection

In [ ]:
# fetch data set
req = requests.get(
    "https://www.internetwache-polizei-berlin.de/vdb/Fahrraddiebstahl.csv"
)

# # write data to csv file
# csv_file = open("fahrraddiebstaehle.csv", "w", encoding="utf-8")
# csv_file.write(req.text)
# csv_file.close()

# load data to pandas df
df = pd.read_csv(StringIO(req.text), delimiter=",", dtype={"LOR": object})


In [ ]:
response = urlopen("https://raw.githubusercontent.com/JJestel/HCDS/main/a1/data/lor.csv")
lor = pd.read_csv(response, delimiter=";", dtype=object)

response = urlopen("https://raw.githubusercontent.com/JJestel/HCDS/main/a1/data/einwohnerbestand_06-2022_plr.csv")
ew = pd.read_csv(response, delimiter=";", dtype={"PLR-ID": object})

In [ ]:
r = requests.get("https://tsb-opendata.s3.eu-central-1.amazonaws.com/lor_planungsgraeume_2021/lor_planungsraeume_2021.geojson")
geojson = json.loads(r.content)

# Data Processing

In [ ]:
df['TATZEIT_ANFANG'] = pd.to_datetime(df['TATZEIT_ANFANG_DATUM'].astype(str) + ' ' + df['TATZEIT_ANFANG_STUNDE'].astype(str) + ':00:00', dayfirst=True)
df['TATZEIT_ENDE'] = pd.to_datetime(df['TATZEIT_ENDE_DATUM'].astype(str) + ' ' + df['TATZEIT_ENDE_STUNDE'].astype(str) + ':00:00', dayfirst=True)
df = df.drop(columns=['TATZEIT_ANFANG_DATUM', 'TATZEIT_ANFANG_STUNDE', 'TATZEIT_ENDE_DATUM', 'TATZEIT_ENDE_STUNDE'])
df['ANGELEGT_AM'] = pd.to_datetime(df['ANGELEGT_AM'], dayfirst=True)
df

In [ ]:
lor = lor.merge(ew, left_on='Planungsraum (Nummer)', right_on='PLR-ID')
lor = lor.drop(columns= ['techn. Schlüssel (UUID)', 'Stand der Daten', 'ausländer', 'PLR-ID', 'weiblich'])
lor["Fläche in qm"] = lor["Fläche in qm"].str.replace(",", ".").astype(float)
int_cols = ['Gesamt', 'unter6', '6-15', '15-18', '18-27', '27-45','45-55', '55-65', '65undmehr']
lor[int_cols] = lor[int_cols].replace('–', -1).astype(int)

In [ ]:
# transform coordinate reference system
transformer = Transformer.from_crs(25833, 4326)

geojson['crs']['properties']['name'] = "urn:ogc:def:crs:EPSG::4326"

for i in range(len(geojson['features'])):
    for j, el in enumerate(geojson['features'][i]['geometry']['coordinates'][0][0]):
        x, y = transformer.transform(*el)
        geojson['features'][i]['geometry']['coordinates'][0][0][j][0] = y
        geojson['features'][i]['geometry']['coordinates'][0][0][j][1] = x


# Data Analysis

### Where are the most bikes stolen?


In [ ]:
# aggregate thefts per Planungsraum
temp = df.groupby(by="LOR").size().to_frame().rename(columns={0: "Diebstaehle"})
lor = lor.merge(temp, left_on="Planungsraum (Nummer)", right_index=True, how="left")
lor['Diebstaehle'] = lor['Diebstaehle'].fillna(0).astype(int)
lor.sort_values(by='Diebstaehle', ascending=False).head(10)

In [ ]:
bins = [0, *[np.quantile(lor.Diebstaehle, x) for x in [0.2, 0.4, 0.6, 0.8, 0.95]], np.max(lor.Diebstaehle)+1]
lor['cat'] = pd.cut(lor.Diebstaehle, bins, right=False)
colors = ['#fee5d9','#fcbba1','#fc9272','#fb6a4a','#de2d26','#a50f15']

fig = px.choropleth(
    lor.sort_values(by='Diebstaehle'),
    geojson=geojson,
    color="cat",
    color_discrete_sequence=colors,
    locations="Planungsraum (Nummer)",
    featureidkey="properties.PLR_ID",
    projection="mercator",
    hover_data={"Planungsraum (Name)":True, "Diebstaehle":True, "Planungsraum (Nummer)":False},
    width=800,
    height=600
)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()


### Total damage amount?

In [ ]:
sns.histplot(df['SCHADENSHOEHE'], kde=True)

In [ ]:
df['SCHADENSHOEHE'].sum()

### What time of the year? / Seasonal trends?

In [ ]:
df_22 = df[df['TATZEIT_ENDE'].dt.year == 2022]

In [ ]:
count_per_kw = (
    df.groupby(by=df_22["TATZEIT_ANFANG"].dt.isocalendar().week)
    .size()
    .to_frame()
    .rename(columns={0: "Diebstaehle"})
)

count_per_kw.index = count_per_kw.index.astype(int)
sns.lineplot(count_per_kw)


### What daytime?

In [ ]:
g = sns.histplot((df["TATZEIT_ENDE"] - df["TATZEIT_ANFANG"]) / pd.Timedelta("1 hour"))
g.set_xlabel("TATZEIT_ENDE - TATZEIT_ANFANG in Stunden")


In [ ]:
count_per_hour = (
    df.groupby(by=df_22["TATZEIT_ANFANG"].dt.hour)
    .size()
    .to_frame()
    .rename(columns={0: "Diebstaehle"})
)

count_per_hour.index = count_per_hour.index.astype(int)
sns.lineplot(count_per_hour)


In [ ]:
count_per_hour = (
    df.groupby(by=df_22["TATZEIT_ENDE"].dt.hour)
    .size()
    .to_frame()
    .rename(columns={0: "Diebstaehle"})
)

count_per_hour.index = count_per_hour.index.astype(int)
sns.lineplot(count_per_hour)
